# Installation
## Necessary Python Modules

1. Install [bottle](https://bottlepy.org)
<pre>
pip install bottle
</pre>

1. Install lorem
<pre>
pip install lorem
</pre>


## Additional resources

1. Crate a free account on [PythonAnywhere.com](http://pythonanywhere.com)
1. Create a new empty repository on [github.com](http://github.com) or [bitbucket.org](http://bitbucket.org)
1. [Optional] Download the **source code** version of [bootstrap](http://getbootstrap.com/getting-started/#download)

# Source code

<pre>
git clone https://github.com/AgataMigalska/pyladiesbottles.git
</pre>

# Live demo

http://pyladiesbottle.pythonanywhere.com

Note: Demo is on a free account which has limited CPU time, so it might happen that the website is no longer respo

# Writing code

## Index

1. Create new package `blog`. Inside the `blog` create a new catalogue `views`. In `views` create new file `index.tpl`. Your directory structure should look like this:
<pre>
blog
    |-- views
        |-- index.tpl
</pre>
1. Paste the code below to `index.tpl`
<pre>
&lt;!DOCTYPE html&gt;
&lt;html&gt;
&lt;head&gt;
&lt;meta charset="utf-8"&gt;
&lt;title&gt;Message in the Bottle&lt;/title&gt;
&lt;/head&gt;
&lt;body&gt;
&lt;h1&gt;Message in the Bottle!&lt;/h1&gt;
&lt;/body&gt;
&lt;/html&gt;
</pre>

1. In `blog` create new module `server.py`

<pre>
import bottle

@bottle.route('/')
def index():
    return bottle.template('index')
    
application = bottle.default_app()
application.run()
</pre>


## Running the server

1. Start the server

1. Open a browser and go to http://localhost:8080

1. Troubleshooting: If you see `Template 'index' not found.` see here: [https://bottlepy.org/docs/dev/faq.html](https://bottlepy.org/docs/dev/faq.html) 

## Adding CSS style

To build a good-looking website we will use [bootstrap](http://getbootstrap.com/getting-started/#download). If you download the source code archive you will find several examples in the `docs/examples` folder. We will be using the `blog` css.

1. In `blog` directory create `static` directory. 
1. Copy `js` and `css` folders from the cloned repository (these are taken from bootstrap + there is an additional style.css)
1. In `index.tpl` in the head section add 
<pre>
&lt;link rel="stylesheet" type="text/css" href="/static/css/style.css"&gt;
</pre>

1. Run the server. Note that you will see 
<pre>
"GET /static/css/style.css HTTP/1.1" 404 736
</pre>

1. In the server.py add a function that will serve static content.

<pre>
import os

@bottle.route('/static/&lt;directory&gt;/&lt;filename&gt;')
def server_static(directory, filename):
    root = os.path.join(os.getcwd(), 'static', directory)
    return bottle.static_file(filename, root=root)
</pre>

## Navigation

Within `server.py` write navigation between pages

<pre>
@bottle.route('/')
def index():
    return bottle.template('index')
    
@bottle.route('/about')
def about():
    return bottle.template('about')
    
@bottle.get('/blog')
def blog():
    return bottle.template('blog')
    
@bottle.get('/post')
def new_entry():
    return bottle.template('post')
    
@bottle.post('/blog')
def add_entry():
    # do stuff
    return blog()
</pre>



### Templates 
1. Copy the contents of template_base.tpl into base.tpl
1. Copy the contents of template_blog.tpl into blog.tpl
1. Create remaining tpl files: post.tpl and about.tpl. Leave them blank for now.

`base.tpl` is a master page for our website. For the content that will be changing put
<pre>
{{!base}}
</pre>

Exclamation mark means that exact html code will be pasted. Without it, html tags are escaped by Bottle.

### Nav header
* In base.tpl change the top navigation to:

<pre>
&lt;nav class="nav blog-nav"&gt;
    % try:
        % if page == 'index':
            &lt;a class="nav-link active" href="/"&gt;Home&lt;/a&gt;
        % else:
            &lt;a class="nav-link" href="/"&gt;Home&lt;/a&gt;
        % end
        % if page == 'blog':
            &lt;a class="nav-link active" href="/blog"&gt;Blog&lt;/a&gt;
        % else:
            &lt;a class="nav-link" href="/blog"&gt;Blog&lt;/a&gt;
        % end
        % if page == 'post':
            &lt;a class="nav-link active" href="/post"&gt;New post&lt;/a&gt;
        % else:
            &lt;a class="nav-link" href="/post"&gt;New post&lt;/a&gt;
        % end
        % if page == 'about':
            &lt;a class="nav-link active" href="/about"&gt;About&lt;/a&gt;
        % else:
            &lt;a class="nav-link" href="/about"&gt;About&lt;/a&gt;
        % end
    % except NameError:
        &lt;a class="nav-link" href="/"&gt;Home&lt;/a&gt;
        &lt;a class="nav-link" href="/blog"&gt;Blog&lt;/a&gt;
        &lt;a class="nav-link" href="/post"&gt;New post&lt;/a&gt;
        &lt;a class="nav-link" href="/about"&gt;About&lt;/a&gt;
    % end
&lt;/nav&gt;
</pre>

* In `server.py` send an information about page to templates

<pre>
@bottle.route('/')
def index():
    return bottle.template('index', page='index')
    
@bottle.route('/about')
def about():
    return bottle.template('about', page='about')
    
@bottle.get('/blog')
def blog():
    with get_dao() as data_access_object:
        posts = data_access_object.select()
    return bottle.template('blog', posts=posts, page='blog')
    
@bottle.get('/post')
def new_entry():
    return bottle.template('post', page='post')
</pre>

## List blog entries

* Copy dao.py to your project
* Update blog() method to use the dao

<pre>
@bottle.get('/blog')
def blog():
    with get_dao() as data_access_object:
        posts = data_access_object.select()
    return bottle.template('blog', posts=posts, page='blog')
       
def get_dao():
    import dao
    return dao.MockDataAccessObject()
</pre>

* Update blog.tpl to display our posts
<pre>
%for idx, post in enumerate(posts):
    &lt;div class="blog-post"&gt;
        &lt;h2 class="blog-post-title"&gt;{{post['TITLE']}}&lt;/h2&gt;
        &lt;p class="blog-post-meta"&gt;{{post['POST_DATE']}} by &lt;a href="#"&gt;{{post['AUTHOR']}}&lt;/a&gt;&lt;/p&gt;
        &lt;p&gt;{{post['CONTENT']}}&lt;/p&gt;
    &lt;/div&gt;
%end
</pre>

## Add new blog entry
In blog.tpl add a button that allows to add a new entry:
<pre>
&lt;div class="blog-pagination"&gt;
    &lt;a class="btn btn-primary" href="/post"&gt;Add new entry&lt;/a&gt;
&lt;/div&gt;
</pre>

In post.tpl paste the following code:
<pre>
% rebase('base.tpl')
&lt;form id="new_post" action="blog" method="post" class="form-new-post"&gt;
    &lt;h2 class="form-heading"&gt;New Message&lt;/h2&gt;
    &lt;label for="title"&gt;Title:&lt;/label&gt;
    &lt;input type="text" class="form-control" name="title" id="title"/&gt;
    &lt;label for="author"&gt;Author:&lt;/label&gt;
    &lt;input type="text" class="form-control" name="author" id="author"/&gt;
    &lt;label for"content"&gt;Content:&lt;/label&gt;
    &lt;textarea name="content" class="form-control" id="content"&gt;&lt;/textarea&gt;
    &lt;button class="btn btn-primary" type="submit" id="submit"&gt;Save&lt;/button&gt;
&lt;/form&gt;
</pre>

### Saving an entry
Change add_entry() function to save a post

<pre>
@bottle.post('/blog')
def add_entry():
    data = dict()
    data['TITLE'] = bottle.request.forms.get('title')
    data['AUTHOR'] = bottle.request.forms.get('author')
    data['CONTENT'] = bottle.request.forms.get('content')
    data['POST_DATE'] = datetime.now()
    
    with get_dao() as data_access_object:
        data_access_object.insert(data)
    return blog()
</pre>

## Git Repository
Push your project to git repository

# Pythonanywhere.com
## Source code and database
1. Log into Pythonanywhere.com. 
1. Set up a password for MySql database (say, notrealpassword)
1. Create a new database called *messages*
1. Start a bash console and clone your git repository
1. In a bash console type (changing the user, password and repositoryname)

<pre>
mysql --user=user --password=password --host=user.mysql.pythonanywhere-services.com 'user$messages' < /home/user/repositoryname/blog/db/create_db.sql
</pre>

<pre>
mysql --user=user --password=password --host=user.mysql.pythonanywhere-services.com 'user$messages' < /home/user/repositoryname/blog/db/test_data.sql
</pre>


## Settings.ini

Create a file `settings.ini` in a `db` folder. Paste in the following information (replace username and password):
<pre>
[mysql]
host=username.mysql.pythonanywhere-services.com
user=username
password=password
database=username$messages
charset=utf8
buffered=False
</pre>

## Virtual Environment
1. Make virtual environment
<pre>
mkvirtualenv messagesenv --python=/usr/bin/python3.6
</pre>
1. pip install 
<pre>
pip install lorem
pip install bottle
pip install mysqlclient
pip install mysql-connector-python
</pre>
1. Go to the Web tab, Virtualenv section, and enter the path: 
<pre>
/home/myusername/.virtualenvs/myvirtualenv
# e.g. /home/pyladiesbottle/.virtualenvs/messagesenv
</pre>


## Webapp configuration
1. On the Web tab create new Bottle application. Your project home should be
<pre>
project_home = u'/home/user/repositoryname/blog'
</pre>
1. Click on the link leading to your website and check that "Hello from bottle is displayed"
1. Go to username_pythonanywhere_com_wsgi.py file and change the last line to
<pre>
from server import application
</pre>
1. Save, Close, Reload your website